In [1]:
from ltgee import LandTrendr
from datetime import date
import geemap
import ee
import os

# Authenticate with Google Earth Engine before using. See link below for more information:
# https://developers.google.com/earth-engine/guides/auth

# add in your own project name
ee.Initialize()

In [2]:
# Example arguments to pass to the LandTrendr class. See docstring of LandTrendr for more information
lt_params = {
    "start_date": date(1985, 6,1),
    "end_date": date(2017, 9,1),
    "index": 'NBR',
    "ftv_list": ['TCB', 'TCG', 'TCW', 'NBR'],
    "mask_labels": ['cloud', 'shadow', 'snow', 'water'],

    "area_of_interest": ee.Geometry({
        'type': 'Polygon',
        'coordinates': [
            [
                [-122.37202331327023,44.62585686599272],
                [-122.26765319608273,44.62585686599272],
                [-122.26765319608273,44.696185837887384],
                [-122.37202331327023,44.696185837887384],
                [-122.37202331327023,44.62585686599272],
                ]
            ]
    }),
    "run_params": {
            "maxSegments": 6,
            "spikeThreshold": 0.9,
            "vertexCountOvershoot":  3,
            "preventOneYearRecovery":  True,
            "recoveryThreshold":  0.25,
            "pvalThreshold":  .05,
            "bestModelProportion":  0.75,
            "minObservationsNeeded": 6,
        }
}

In [3]:
# Instantiating LandTrendr object. Note: the LandTrendr algorithm will be run greedily on GEE (ie the script will immediately request to run the algorithm on GEE as soon as it has all the collections built unless run=False).
lt = LandTrendr(**lt_params)
lt.data

In [4]:
# Instantiating Landtrendr class with debug parameter as True (default: False). Additional properties will be available using this method.
lt_debug = LandTrendr(debug=True, **lt_params)
lt_debug.sr_collection, lt_debug.lt_collection, lt_debug.clear_pixel_count_collection

(<ee.imagecollection.ImageCollection at 0x1796dc6d750>,
 <ee.imagecollection.ImageCollection at 0x1796dc6d2d0>)

In [5]:
# The LandTrendr algorithm will NOT be run unless the run(debug=False) method is called after.
# Note: The previously built collections will be used to run the algorithm unless class attributes have changed.
lt_debug.run()
lt_debug.data

In [6]:
# You may reset different attributes, however, you must rerun LandTrendr using the run method to get the updated lt data.
lt_debug.ftv_list = ['B7']
lt_debug.run(debug=True)
lt_debug.run()
lt_debug.data, lt_debug.sr_collection, lt_debug.lt_collection, lt_debug.clear_pixel_count_collection

(<ee.image.Image at 0x1796df893d0>,
 <ee.imagecollection.ImageCollection at 0x1796df570d0>)

In [7]:
# You may call lt.run(debug=True) to rebuild the collections and collect them in attributes at any time.
# Note: Keep in mind the LandTrendr algorithm will not be run again unless lt.run(debug=False) is called.
lt.run(debug=True)
lt.sr_collection, lt.lt_collection, lt.clear_pixel_count_collection

(<ee.imagecollection.ImageCollection at 0x1796e1bc090>,
 <ee.imagecollection.ImageCollection at 0x1796e1d56d0>)

In [8]:
# Displaying collection information in jupyter notebook
# Some data requires some pre-processing. There are some methods for that (see below/documentation).
lt.lt_collection

In [9]:
# Reruning without debug

lt.run()

In [10]:
# Example 1: Displaying images from Landtrendr

lt_tcb = lt.data.select('ftv_nbr_fit').clip(lt_params['area_of_interest'])
map = geemap.Map()
map.centerObject(lt_params['area_of_interest'], 13)
map.addLayer(lt_tcb, {}, 'lt_nbr')
map.addLayer(lt_params['area_of_interest'], {}, 'area_of_interest')
map

Map(center=[44.661026124368775, -122.31983825467653], controls=(WidgetControl(options=['position', 'transparen…

In [11]:
# Example 2: Displaying collection to band stack

# Note: lt_collection is already transformed and saved using debug=True
collection_to_band_stack = lt.collection_to_band_stack(lt.lt_collection)

map = geemap.Map()
map.centerObject(lt_params['area_of_interest'], 13)
# You know I wished I picked a more interesting area...maybe zoom out
map.addLayer(collection_to_band_stack, {"bands":["2000"],"min":-100,"max":1000,"palette":["ff2f0d","fff825","0ab308"]})
map.addLayer(lt_params['area_of_interest'], {}, 'area_of_interest')
map

Map(center=[44.661026124368775, -122.31983825467653], controls=(WidgetControl(options=['position', 'transparen…

In [12]:
# Example 3: Displaying change data
# Note: Change params does not require all keys to be present and will only process what is passed in.

change_params = {
                    'delta': 'loss',
                    'sort': 'greatest',
                    'years': {'start': lt_params["start_date"].year + 1, 'end': lt_params["end_date"].year},
                    'mag': {'value': 200, 'operator': '>' },
                    'dur': {'value': 4, 'operator': '<'},
                    'preval': {'value': 300, 'operator': '>'},
                    'mmu': {'value': 5}
                }
change_image = lt.get_change_map(change_params)

palette = ['#9400D3', '#4B0082', '#0000FF', '#00FF00', '#FFFF00', '#FF7F00', '#FF0000'];
yod_vis_params = {
  'min': change_params['years']['start'],
  'max': change_params['years']['end'],
  'palette': palette
}
mag_vis_params = {
  'min': 200,
  'max': 800,
  'palette': palette
}


map = geemap.Map()
map.centerObject(lt_params['area_of_interest'], 13)
map.addLayer(change_image.select(['mag']), yod_vis_params, 'Magnitude of Change')
map.addLayer(change_image.select(['yod']), mag_vis_params, 'Year of Detection')
map.addLayer(lt_params['area_of_interest'], {}, 'area_of_interest')
map

Map(center=[44.661026124368775, -122.31983825467653], controls=(WidgetControl(options=['position', 'transparen…

In [13]:
# Example 4: Displaying segment data
# In the top right corner hover over the wrench and click the 'i' to use the inspector. Click any point in the aoi to see values.

seg_info = lt.get_segment_data(lt_params['index'], 'all').clip(lt_params['area_of_interest'])
map = geemap.Map()
map.centerObject(lt_params['area_of_interest'], 13)
map.addLayer(seg_info, {}, 'seg_info')
map.addLayer(lt_params['area_of_interest'], {}, 'area_of_interest')
map

Map(center=[44.661026124368775, -122.31983825467653], controls=(WidgetControl(options=['position', 'transparen…

In [14]:
# Example 5: Displaying rgb image

vis_params = {
  'min': [604, -49, -2245],
  'max': [5592, 3147, 843],
  'gamma': [1, 1, 1]
}

lt_rgb = lt.get_fitted_rgb_col(['TCB', 'TCG', 'TCW'], vis_params)
map = geemap.Map()
map.centerObject(lt_params['area_of_interest'], 13)
map.addLayer(lt_rgb, {}, 'lt_rgb')
map.addLayer(lt_params['area_of_interest'], {}, 'area_of_interest')
map

Map(center=[44.661026124368775, -122.31983825467653], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
# Example 6: Creating a gif using data from landtrendr

vid_params = {
  'crs': 'EPSG:3857',
  'dimensions': 300,
  'region': lt_params['area_of_interest'],
  'framesPerSecond': 8
}
print(lt_rgb.getVideoThumbURL(vid_params))